# Imports

In [ ]:
import itertools as it
import os
import re
from collections import namedtuple
from functools import partial
from pathlib import Path

import dask
import distributed
import h5py
import holoviews as hv
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import skimage.measure
import zarr
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from tqdm.auto import tqdm

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.image_analysis.new as new
from paulssonlab.image_analysis import *

In [ ]:
# %load_ext pyinstrument

In [ ]:
hv.extension("bokeh")

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="06:00:00",
    memory="2GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(1)

In [ ]:
cluster.adapt(maximum=20)

# Metrics

In [ ]:
nd2 = nd2reader.ND2Reader(
    "/home/jqs1/scratch/jqs1/microscopy/230130/230130_lane3_50ms_refocused001.nd2"
)

In [ ]:
nd2.sizes

In [ ]:
img = nd2.get_frame_2D(c=0)

In [ ]:
imgs = [nd2.get_frame_2D(c=c) for c in range(4)]

In [ ]:
plt.imshow(img)

In [ ]:
hv.Raster(imgs[0]).opts(aspect="equal")

In [ ]:
hv.Raster(imgs[0]).opts(aspect="equal") + hv.Raster(imgs[2]).opts(aspect="equal")

In [ ]:
crop = imgs[0][1300:1700, 1900:2400]

In [ ]:
plt.imshow(skimage.filters.roberts(crop))

In [ ]:
plt.imshow(skimage.filters.sato(crop, black_ridges=False))

In [ ]:
%%time
x = skimage.filters.sato(img, black_ridges=False)

In [ ]:
hv.Raster(x).opts(aspect="equal")

In [ ]:
plt.imshow()

In [ ]:
plt.imshow(skimage.filters.sato(crop, sigmas=range(1, 4, 2), black_ridges=False))

In [ ]:
plt.imshow(skimage.filters.sato(crop, sigmas=range(1, 6, 1), black_ridges=False))

In [ ]:
plt.imshow(skimage.filters.sato(crop, sigmas=range(1, 4, 1), black_ridges=False))

In [ ]:
plt.imshow(skimage.filters.sato(crop, black_ridges=False))

In [ ]:
plt.imshow(crop)

In [ ]:
plt.imshow(
    skimage.filters.butterworth(crop, cutoff_frequency_ratio=0.04, high_pass=True)
)

In [ ]:
%%time
img_highpass = skimage.filters.butterworth(
    img, cutoff_frequency_ratio=0.04, high_pass=True
)

In [ ]:
hv.Raster(img_highpass).opts(aspect="equal")

In [ ]:
%%time
y = skimage.filters.sato(img_highpass, sigmas=range(1, 4, 1), black_ridges=False)

In [ ]:
hv.Raster(y).opts(aspect="equal")

In [ ]:
%%time
diag = util.tree()
trenches = trench_detection.find_trenches(y, diagnostics=diag)

In [ ]:
ui.show_plot_browser(diag["labeling"]);

In [ ]:
diag["labeling"].keys()